<a href="https://colab.research.google.com/github/debasishsarangi88/S4/blob/master/Debasish_EVA5_Session_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [12]:
# nn.Module is the base class for all neural networks. When we pass this here, 
#we are saying that 'Net' class will inherit the nn.Module class. 

class Net(nn.Module): 
    def __init__(self): #It is a dunder method meant for initializing the values of instance members whenever a class is instantiated 
        super(Net, self).__init__() # super() is a way that allows us to avoid referrring to the base class explicitly. 
        self.conv1 = nn.Conv2d(1, 16, 3 ,padding=1) #InputSize: 28x28x1   OutputSize: 28x28x16  RF: 3x3  
        self.conv2 = nn.Conv2d(16,32, 3  , padding=1)#InputSize: 28x28x16  OutputSize: 28x28x32  RF: 5x5 
        self.pool1 = nn.MaxPool2d(2, 2) #InputSize: 28x28x32  OutputSize: 14x14x32  RF: 10x10(Assumption:- maxpooling doubles the receptive field as of now.) 
        self.conv3 = nn.Conv2d(32,64, 3, padding=1) #InputSize: 14x14x32  OutputSize: 14x14x64 RF: 12x12(Size does not decrease since padding is used)
        self.conv4 = nn.Conv2d(64,128, 3 , padding=1) #InputSize: 14x14x64 OutputSize: 14x14x128 RF: 14x14(Size does not decrease since padding is used)
        self.pool2 = nn.MaxPool2d(2, 2) #InputSize: 14x14x128 OutputSize: 7x7x128   RF: 28x28(Assumption:- maxpooling doubles the receptive field as of now.))
        self.conv5 = nn.Conv2d(128,128, 3)  #InputSize: 7x7x128   OutputSize: 5x5x128   RF: 30x30
        self.conv6 = nn.Conv2d(128,196, 3) #InputSize: 5x5x128   OutputSize: 3x3x196  RF: 32x32   
        self.conv7 = nn.Conv2d(196, 10, 3) #InputSize: 3x3x196  OutputSize: 1x1x10    RF: 34x34

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [13]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
            Conv2d-2           [-1, 32, 28, 28]           4,640
         MaxPool2d-3           [-1, 32, 14, 14]               0
            Conv2d-4           [-1, 64, 14, 14]          18,496
            Conv2d-5          [-1, 128, 14, 14]          73,856
         MaxPool2d-6            [-1, 128, 7, 7]               0
            Conv2d-7            [-1, 128, 5, 5]         147,584
            Conv2d-8            [-1, 196, 3, 3]         225,988
            Conv2d-9             [-1, 10, 1, 1]          17,650
Total params: 488,374
Trainable params: 488,374
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.71
Params size (MB): 1.86
Estimated Total Size (MB): 2.57
-------------------------------------------

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [21]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [15]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [22]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.063, momentum=0.8)

for epoch in range(1, 19):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0543, Accuracy: 9825/10000 (98%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0297, Accuracy: 9900/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0274, Accuracy: 9932/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0278, Accuracy: 9935/10000 (99%)



loss=1.6378269720007665e-05 batch_id=468: 100%|██████████| 469/469 [00:23<00:00, 19.84it/s]



Test set: Average loss: 0.0291, Accuracy: 9937/10000 (99%)

